In [1]:
import sys
import time
sys.path.insert(0, '/home/anish/ASC_ML')

In [2]:
from ASC_ML import multiple_model_gen_v3 as multiple
from ASC_ML import dataframe_extractor as de
from ASC_ML import model_generation as model_gen
# from ASC_ML import model_gen_train_test as testing
from dask_ml.preprocessing import LabelEncoder
# from dask_ml.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

2022-08-12 12:05:03.466169: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
# m = multiple.Multiple_Model_Gen(max_no_layers = 2, model_per_batch = 10)

In [4]:
# import dask.dataframe as dd
# import pandas as pd
# data = dd.from_pandas(pd.Series(['a', 'a', 'b'], dtype='category'),
#                      npartitions=1)
# le = LabelEncoder()
# le.fit_transform(data).compute()
# print(type(le.fit_transform(data).compute()))

In [5]:
#For House Prediction Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=train.csv
directory1 = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv"


# returns list of dask dataframe [singular_df_x, singular_df_y] or [train_df_x, train_df_y, test_df_x, test_df_y]
dataset_list = de.DataframeExtractor_csv(directory1, label_names = ["SalePrice"]).df_list

print("Train Dataset X Columns")
dataset_list[0].head()

Reading single csv from /home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv
Train Dataset X Columns


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,12.0,2008.0,WD,Normal


In [6]:
dataset_list[1].head()

,SalePrice
0,208500.0
1,181500.0
2,223500.0
3,140000.0
4,250000.0


In [7]:
train_x = dataset_list[0].copy()

In [8]:
train_x = train_x.fillna(0)

In [9]:
train_x.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [10]:
train_x['MSSubClass'] = train_x['MSSubClass'].apply(str)

train_x['OverallCond'] = train_x['OverallCond'].astype(str)

train_x['YrSold'] = train_x['YrSold'].astype(str)
train_x['MoSold'] = train_x['MoSold'].astype(str)
train_x.head()

/home/anish/anaconda3/envs/tfgpu/lib/python3.9/site-packages/dask/dataframe/core.py:3946: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('MSSubClass', 'object'))

  warnings.warn(meta_warning(meta))


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [11]:
# a = train_x['LotShape']

In [12]:
# le = LabelEncoder() 
# le.fit_transform(a).compute()
# train_x['LotShape'] = le.fit_transform(train_x['LotShape'])

In [13]:
# train_x.head()

In [14]:
def keyval(allkeys):
    i = 0
    keyvalpairs = list()
    for key in allkeys:
        keyvalpairs.append((key,i))
        i+=1
    return dict(keyvalpairs)

In [15]:
# cols = ['FireplaceQu', 'LotShape']
cols = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold']
keys = {}
for col in cols:
    key = train_x[col].unique()
    keyvalpairs = keyval(key)
    keys.update({col:keyvalpairs})


In [16]:
keys

{'FireplaceQu': {0: 0, 'TA': 1, 'Gd': 2, 'Fa': 3, 'Ex': 4, 'Po': 5},
 'BsmtQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 0: 3, 'Fa': 4},
 'BsmtCond': {'TA': 0, 'Gd': 1, 0: 2, 'Fa': 3, 'Po': 4},
 'GarageQual': {'TA': 0, 'Fa': 1, 'Gd': 2, 0: 3, 'Ex': 4, 'Po': 5},
 'GarageCond': {'TA': 0, 'Fa': 1, 0: 2, 'Gd': 3, 'Po': 4, 'Ex': 5},
 'ExterQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'ExterCond': {'TA': 0, 'Gd': 1, 'Fa': 2, 'Po': 3, 'Ex': 4},
 'HeatingQC': {'Ex': 0, 'Gd': 1, 'TA': 2, 'Fa': 3, 'Po': 4},
 'PoolQC': {0: 0, 'Ex': 1, 'Fa': 2, 'Gd': 3},
 'KitchenQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'BsmtFinType1': {'GLQ': 0,
  'ALQ': 1,
  'Unf': 2,
  'Rec': 3,
  'BLQ': 4,
  0: 5,
  'LwQ': 6},
 'BsmtFinType2': {'Unf': 0,
  'BLQ': 1,
  0: 2,
  'ALQ': 3,
  'Rec': 4,
  'LwQ': 5,
  'GLQ': 6},
 'Functional': {'Typ': 0,
  'Min1': 1,
  'Maj1': 2,
  'Min2': 3,
  'Mod': 4,
  'Maj2': 5,
  'Sev': 6},
 'Fence': {0: 0, 'MnPrv': 1, 'GdWo': 2, 'GdPrv': 3, 'MnWw': 4},
 'BsmtExposure': {'No': 0, 'Gd': 1, 'Mn': 2, '

In [17]:
train_X = train_x.replace(keys).copy()

In [18]:
train_X['FireplaceQu'].unique().compute()

0    0
1    1
2    2
3    3
4    4
5    5
Name: FireplaceQu, dtype: int64

In [19]:
train_X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,0,RL,65.0,8450.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,0,WD,Normal
1,2.0,1,RL,80.0,9600.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,1,1,WD,Normal
2,3.0,0,RL,68.0,11250.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2,0,WD,Normal
3,4.0,2,RL,60.0,9550.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,2,WD,Abnorml
4,5.0,0,RL,84.0,14260.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,3,0,WD,Normal


In [20]:
cols_to_remove = []
for col in train_X.columns:
#     print(type(col))
    try:
        a = train_X[col].astype(float).compute()
    except ValueError:
#         print('Couldn\'t covert %s to float' % col)
        cols_to_remove.append(col)
        pass

# keep only the columns in df that do not contain string
train_X = train_X[[col for col in train_X.columns if col not in cols_to_remove]]

In [21]:
print(cols_to_remove)

['MSZoning', 'LandContour', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition']


In [22]:
train_X = train_X.drop(["Id"], axis = 1)

In [23]:
# tsklearn.preprocessing.MinMaxScalerrain_X.head()

In [24]:
len(train_X.columns)

58

In [25]:
len(dataset_list[1].columns)

1

In [26]:
scaler_x = MinMaxScaler()
scaler_x.fit(train_X)
train_X_scaled = scaler_x.transform(train_X)

scaler_y = MinMaxScaler()
scaler_y.fit(dataset_list[1])
train_Y_scaled = scaler_y.transform(dataset_list[1])

In [27]:
# col = train_X.columns.to_list()
# scaler_d = MinMaxScaler()
# train_X.columns = list(train_X.columns)
# scaled_train_X = scaler_d.fit_transform(train_X[col])

In [28]:
train_X_scaled
# train_Y_scaled

array([[0.        , 0.20766773, 0.0334198 , ..., 0.        , 0.        ,
        0.        ],
       [0.07142857, 0.25559105, 0.03879502, ..., 0.        , 0.09090909,
        0.25      ],
       [0.        , 0.2172524 , 0.04650728, ..., 0.        , 0.18181818,
        0.        ],
       ...,
       [0.14285714, 0.21086262, 0.03618687, ..., 0.16129032, 0.09090909,
        1.        ],
       [0.07142857, 0.2172524 , 0.03934189, ..., 0.        , 0.63636364,
        1.        ],
       [0.07142857, 0.23961661, 0.04037019, ..., 0.        , 1.        ,
        0.        ]])

In [29]:
train_Y_scaled

array([[0.24107763],
       [0.20358284],
       [0.26190807],
       ...,
       [0.321622  ],
       [0.14890293],
       [0.15636717]])

In [30]:
print(train_X_scaled.shape)
print(train_Y_scaled.shape)

(1460, 58)
(1460, 1)


In [31]:
import numpy as np
total_data = np.append(train_X_scaled, train_Y_scaled, axis = 1)

In [32]:
print(total_data.shape)

(1460, 59)


In [33]:
from sklearn.model_selection import train_test_split
(train, test) = train_test_split(total_data, test_size=0.25, random_state=42)

In [34]:
trainX = train[:,:58]
trainY = train[:,-1]
testX = test[:,:58]
testY = test[:,-1]
print(trainX.shape)
print(trainY.shape)

(1095, 58)
(1095,)


In [35]:
train_Y_scaled

array([[0.24107763],
       [0.20358284],
       [0.26190807],
       ...,
       [0.321622  ],
       [0.14890293],
       [0.15636717]])

In [36]:
trainY = trainY.reshape((trainY.shape[0],1))
print(trainY.shape)

(1095, 1)


In [37]:
testY = testY.reshape((testY.shape[0],1))

## 2,3 Hidden Layers, 100 epochs, time taken = 147s (model saved)

In [38]:
start = time.time()
m = multiple.Multiple_Model_Gen_V3(trainX, trainY, testX, testY, 10, 128, input_shape = 58, 
                                   max_no_layers = 3, model_per_batch = 10, 
                                   save_dir = "/home/anish/ASC_ML_test_weights/")
m.get_model_confs()
m.get_best_models()
end = time.time()
print("Time Taken : ", (end-start))

2022-08-12 12:05:18.598365: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-12 12:05:18.599706: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-12 12:05:18.668972: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-12 12:05:18.669271: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.695GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2022-08-12 12:05:18.669308: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-08-12 12:05:18.672152: I tensorflow/stream_executor/platform/de

Epoch 1/10
9/9 [==============================] - 2s 5ms/step - loss: 2.8045 - output_layer_dense_16_16_loss: 0.1512 - output_layer_dense_16_32_loss: 0.2114 - output_layer_dense_16_64_loss: 0.3138 - output_layer_dense_16_128_loss: 0.1810 - output_layer_dense_16_196_loss: 0.2467 - output_layer_dense_16_256_loss: 0.2658 - output_layer_dense_32_16_loss: 0.6010 - output_layer_dense_32_32_loss: 0.4143 - output_layer_dense_32_64_loss: 0.3092 - output_layer_dense_32_128_loss: 0.1103
Epoch 2/10
1/9 [==>...........................] - ETA: 0s - loss: 1.4479 - output_layer_dense_16_16_loss: 0.0919 - output_layer_dense_16_32_loss: 0.1427 - output_layer_dense_16_64_loss: 0.1824 - output_layer_dense_16_128_loss: 0.1033 - output_layer_dense_16_196_loss: 0.0955 - output_layer_dense_16_256_loss: 0.1595 - output_layer_dense_32_16_loss: 0.3390 - output_layer_dense_32_32_loss: 0.1280 - output_layer_dense_32_64_loss: 0.1170 - output_layer_dense_32_128_loss: 0.0887

2022-08-12 12:05:20.719489: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


9/9 [==============================] - 0s 4ms/step - loss: 1.4245 - output_layer_dense_16_16_loss: 0.1104 - output_layer_dense_16_32_loss: 0.1350 - output_layer_dense_16_64_loss: 0.1779 - output_layer_dense_16_128_loss: 0.1032 - output_layer_dense_16_196_loss: 0.1099 - output_layer_dense_16_256_loss: 0.1311 - output_layer_dense_32_16_loss: 0.2964 - output_layer_dense_32_32_loss: 0.1478 - output_layer_dense_32_64_loss: 0.1281 - output_layer_dense_32_128_loss: 0.0849
Epoch 3/10
9/9 [==============================] - 0s 4ms/step - loss: 1.0487 - output_layer_dense_16_16_loss: 0.0971 - output_layer_dense_16_32_loss: 0.1078 - output_layer_dense_16_64_loss: 0.1230 - output_layer_dense_16_128_loss: 0.0767 - output_layer_dense_16_196_loss: 0.0820 - output_layer_dense_16_256_loss: 0.0870 - output_layer_dense_32_16_loss: 0.1972 - output_layer_dense_32_32_loss: 0.1229 - output_layer_dense_32_64_loss: 0.0878 - output_layer_dense_32_128_loss: 0.0671
Epoch 4/10
9/9 [==============================] -

9/9 [==============================] - 0s 4ms/step - loss: 0.5013 - output_layer_dense_32_196_loss: 0.0503 - output_layer_dense_32_256_loss: 0.0509 - output_layer_dense_64_16_loss: 0.0522 - output_layer_dense_64_32_loss: 0.0559 - output_layer_dense_64_64_loss: 0.0521 - output_layer_dense_64_128_loss: 0.0455 - output_layer_dense_64_196_loss: 0.0491 - output_layer_dense_64_256_loss: 0.0464 - output_layer_dense_128_16_loss: 0.0555 - output_layer_dense_128_32_loss: 0.0436
Epoch 8/10
9/9 [==============================] - 0s 4ms/step - loss: 0.4931 - output_layer_dense_32_196_loss: 0.0504 - output_layer_dense_32_256_loss: 0.0458 - output_layer_dense_64_16_loss: 0.0527 - output_layer_dense_64_32_loss: 0.0588 - output_layer_dense_64_64_loss: 0.0482 - output_layer_dense_64_128_loss: 0.0454 - output_layer_dense_64_196_loss: 0.0495 - output_layer_dense_64_256_loss: 0.0460 - output_layer_dense_128_16_loss: 0.0528 - output_layer_dense_128_32_loss: 0.0435
Epoch 9/10
9/9 [===========================

...

9/9 [==============================] - 0s 5ms/step - loss: 0.4177 - output_layer_dense_196_32_32_loss: 0.0445 - output_layer_dense_196_64_16_loss: 0.0442 - output_layer_dense_196_64_32_loss: 0.0403 - output_layer_dense_196_64_64_loss: 0.0436 - output_layer_dense_196_128_16_loss: 0.0421 - output_layer_dense_196_128_32_loss: 0.0421 - output_layer_dense_196_128_64_loss: 0.0417 - output_layer_dense_196_128_128_loss: 0.0404 - output_layer_dense_196_196_16_loss: 0.0403 - output_layer_dense_196_196_32_loss: 0.0382
Epoch 8/10
9/9 [==============================] - 0s 5ms/step - loss: 0.3689 - output_layer_dense_196_32_32_loss: 0.0399 - output_layer_dense_196_64_16_loss: 0.0387 - output_layer_dense_196_64_32_loss: 0.0364 - output_layer_dense_196_64_64_loss: 0.0378 - output_layer_dense_196_128_16_loss: 0.0377 - output_layer_dense_196_128_32_loss: 0.0359 - output_layer_dense_196_128_64_loss: 0.0375 - output_layer_dense_196_128_128_loss: 0.0363 - output_layer_dense_196_196_16_loss: 0.0357 - output

9/9 [==============================] - 0s 4ms/step - loss: 0.3116 - output_layer_dense_196_196_64_loss: 0.0328 - output_layer_dense_196_196_128_loss: 0.0323 - output_layer_dense_196_196_196_loss: 0.0285 - output_layer_dense_196_196_256_loss: 0.0329 - output_layer_dense_196_256_16_loss: 0.0326 - output_layer_dense_196_256_32_loss: 0.0345 - output_layer_dense_196_256_64_loss: 0.0271 - output_layer_dense_196_256_128_loss: 0.0326 - output_layer_dense_196_256_196_loss: 0.0300 - output_layer_dense_196_256_256_loss: 0.0283

 
 

loss  :  0.3176022469997406 , TEST :  0.45928698778152466
output_layer_dense_196_196_64_loss  :  0.03231947496533394 , TEST :  0.046657975763082504
output_layer_dense_196_196_128_loss  :  0.03041069768369198 , TEST :  0.04485555738210678
output_layer_dense_196_196_196_loss  :  0.033842284232378006 , TEST :  0.04863782972097397
output_layer_dense_196_196_256_loss  :  0.02892419882118702 , TEST :  0.04227440059185028
output_layer_dense_196_256_16_loss  :  0.031857222318

Epoch 2/10
9/9 [==============================] - 0s 5ms/step - loss: 0.7075 - output_layer_dense_256_196_16_loss: 0.0721 - output_layer_dense_256_196_32_loss: 0.0617 - output_layer_dense_256_196_64_loss: 0.0634 - output_layer_dense_256_196_128_loss: 0.0642 - output_layer_dense_256_196_196_loss: 0.0744 - output_layer_dense_256_256_16_loss: 0.0702 - output_layer_dense_256_256_32_loss: 0.0680 - output_layer_dense_256_256_64_loss: 0.0758 - output_layer_dense_256_256_128_loss: 0.0810 - output_layer_dense_256_256_196_loss: 0.0767
Epoch 3/10
9/9 [==============================] - 0s 5ms/step - loss: 0.5168 - output_layer_dense_256_196_16_loss: 0.0523 - output_layer_dense_256_196_32_loss: 0.0458 - output_layer_dense_256_196_64_loss: 0.0527 - output_layer_dense_256_196_128_loss: 0.0510 - output_layer_dense_256_196_196_loss: 0.0541 - output_layer_dense_256_256_16_loss: 0.0530 - output_layer_dense_256_256_32_loss: 0.0499 - output_layer_dense_256_256_64_loss: 0.0523 - output_layer_dense_256_256_1

9/9 [==============================] - 0s 1ms/step - loss: 0.0197

 
 

loss  :  0.01630372740328312 , TEST :  0.042362313717603683
Epoch 1/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0416
Epoch 2/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0215
Epoch 3/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0162
Epoch 4/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0146
Epoch 5/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0123
Epoch 6/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0134
Epoch 7/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0106
Epoch 8/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0098
Epoch 9/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0108
Epoch 10/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0115

 
 

loss  :  0.010257869958877563 , TEST :  0.04373817890882492
Epoch 1/10
9/9 [===============

9/9 [==============================] - 0s 1ms/step - loss: 0.0051
Epoch 8/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0059
Epoch 9/10
9/9 [==============================] - 0s 993us/step - loss: 0.0059
Epoch 10/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0069

 
 

loss  :  0.005741485394537449 , TEST :  0.0437917485833168
Epoch 1/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0249
Epoch 2/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0163
Epoch 3/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0092
Epoch 4/10
9/9 [==============================] - 0s 973us/step - loss: 0.0075
Epoch 5/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0070
Epoch 6/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0057
Epoch 7/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0057
Epoch 8/10
9/9 [==============================] - 0s 1ms/step - loss: 0.0075
Epoch 9/10
9/9 [=

In [39]:
m.evaluate_dict_list

[{'model_name': 'dense_256_256_128',
  'score': 0.022877873852849007,
  'path_weights': '/home/anish/ASC_ML_test_weights/dense_256_256_128',
  'model': <tensorflow.python.keras.engine.functional.Functional at 0x7f4b386df2e0>},
 {'model_name': 'dense_256_196_32',
  'score': 0.024285905063152313,
  'path_weights': '/home/anish/ASC_ML_test_weights/dense_256_196_32',
  'model': <tensorflow.python.keras.engine.functional.Functional at 0x7f4b5c562910>},
 {'model_name': 'dense_256_256_32',
  'score': 0.02496623620390892,
  'path_weights': '/home/anish/ASC_ML_test_weights/dense_256_256_32',
  'model': <tensorflow.python.keras.engine.functional.Functional at 0x7f4b386da790>},
 {'model_name': 'dense_256_196_128',
  'score': 0.026055162772536278,
  'path_weights': '/home/anish/ASC_ML_test_weights/dense_256_196_128',
  'model': <tensorflow.python.keras.engine.functional.Functional at 0x7f4b386d1670>},
 {'model_name': 'dense_256_196_196',
  'score': 0.026088345795869827,
  'path_weights': '/home/an

In [40]:
a = [1,2,3,4]
b = [5,6,7,8]
c = []
c[0] = a
c[1] = b
print(c)

IndexError: list assignment index out of range